In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

# This is our own tokenizer
from nltk.tokenize import RegexpTokenizer

from docx import Document

# set wd
wd = '/Users/Felicia/Documents/ComputationalLaw/'

# import txt file
with open('WIWpages.txt','r') as f_open:
    data = f_open.read()

# print(data)


In [2]:
# separate data into paragraphs, so each paragraph is 1 case

data2 = re.split('\s{3,}',data) # This uses 3 consecutive whitespace chars as paragraph delimiter.
len(data2)

23

In [3]:
# create empty dfs to fill
columns = ['case_number', 'case_name', 'date', 'state', 'county', 
           'amount', 'pertinent_info',
           'plaintiff', 'defendant', 
           'injury_list', 'event_narrative', 
           'mediator', 'judge', 'experts','attorneys']

finaldf = pd.DataFrame(index=range(0,len(data2)), columns=columns)

columns2 = ['case_name','abdominal', 'ankle', 'arm', 'back', 'brain', 'burn', 'bowel', 'chest', 'circulatory', 
            'ear', 'elbow', 'eye', 'face', 'foot', 'leg', 'genital', 'hand', 'head', 'heart', 
            'hip', 'knee', 'liver', 'mouth', 'neck', 'nose', 'pelvic', 'shoulder', 'spinal', 
            'thigh', 'whole_body', 'wrist', 'pancreas', 'psychological', 'death']

injury_df = pd.DataFrame(index=range(0,len(data2)), columns=columns2)

# use case_name to merge dfs because sometimes case number isn't published

In [4]:
# fill in finaldf (excludes injuries)

# !!!! still missing county
# !!!! multiple plaintiff?
# !!!! date is inconsistent

for x in range(0, len(data2)):
    a = data2[x]
    words = a.split()
    
    if words[0] !='AMOUNT:': #if the first word isn't AMOUNT, it's not a case but page number etc.
        finaldf.loc[x] = "NA" #fill entire row with NA
        
    else: #if it's a case
        lines = a.splitlines() #split the paragraph into lines
        # print(x)
        for i in range(0, len(lines)): 
            line = lines[i]
            w = str.lower(lines[i]).split() #convert text to lowercase and split into words
            
            if w[0] == 'amount:':
                finaldf.loc[x, 'amount'] = w[1].replace('$', '')
                finaldf.loc[x, 'state'] = w[-1].replace(')', '').replace('(', '') 
            if w[0] == 'case' and w[1] == 'number:':
                finaldf.loc[x, 'case_number'] = ' '.join(w[2:len(w)])             
            if w[0] == 'case:':
                finaldf.loc[x, 'case_name'] = ' '.join(w[1:len(w)])
                
                # Plaintiff and defendant
                result= re.search(': *(.*?), *[0-9]{4}', line)
                plt_v_def= result.group(1).split(' v. ')
                plaintiff= plt_v_def[0]
                defendant= plt_v_def[1]
                finaldf.loc[x, 'plaintiff']= plaintiff
                finaldf.loc[x, 'defendant']= defendant
                
                # Date
                # Sometimes it's the last three words
#                 finaldf.loc[x, 'date'] = ' '.join(w[-2:]).replace(')', '').replace('(', '')  #join last two words
                date = line[line.rfind('(')+1:line.rfind(')')]
                finaldf.loc[x, 'date'] = date
            if w[0] == 'mediator:':
                finaldf.loc[x, 'mediator'] = ' '.join(w[1:len(w)])                
            if w[0] == 'judge:':
                finaldf.loc[x, 'judge'] = ' '.join(w[1:len(w)])
            if w[0] == 'attorneys:':
                finaldf.loc[x, 'attorneys'] = ' '.join(w[1:len(w)])
            if w[0] == 'experts:':
                finaldf.loc[x, 'experts'] = ' '.join(w[1:len(w)])                
            if w[0] == 'event:':
                finaldf.loc[x, 'event_narrative'] = ' '.join(w[1:len(w)])
            if w[0] == 'specific' and w[1] == 'injury:':
                finaldf.loc[x, 'injury_list'] = ' '.join(w[2:len(w)])
            if w[0] == 'pertinent' and w[1] == 'information:':     
                finaldf.loc[x, 'pertinent_info'] = ' '.join(w[2:len(w)])
    

In [5]:
finaldf

,case_number,case_name,date,state,county,amount,pertinent_info,plaintiff,defendant,injury_list,event_narrative,mediator,judge,experts,attorneys
0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,cv-2007-000134,"jeremy freeman v. southern medical group, 2014...",October 2014,al,NaN,"1,150,000",NaN,Jeremy Freeman,Southern Medical Group,cardiac arrest; punctured abdominal wall; death,the plaintiff alleged that the defendant plast...,NaN,jerry k. selman',"wayne ross, forensic pathology, lancaster, pa,...","dennis e. goldasich, jr., joshua m. vick and j..."
3,not published,"plaintiff v. defendant doctor, 2016 medical li...",January 2016,ca,NaN,"1,000,000",NaN,Plaintiff,Defendant Doctor,injury to common bile duct and hepatic artery,the plaintiff alleged that the defendant injur...,NaN,not published,"catherine m. graves, m.b.a., economist, fuller...","jeffrey a. milman of hodes milman, irvine, ca,..."
4,3:09-cv-00026,"jackson v. usa, 2012 medical litig. alert lexi...",July 2012,fl,NaN,"750,000",the case was settled prior to trial. the plain...,Jackson,USA,peritonitis; sepsis; stroke; permanent cogniti...,the plaintiff went the emergency room of the d...,NaN,"timothy j. corrigan,",not published,"sean b. cronin of cronin & maxwell, jacksonvil..."
5,2008 ca 028502,"todd rozzo, valerie m. rozzo and their minor c...","Aug. 21, 2012",fl,NaN,"320,000",NaN,"Todd Rozzo, Valerie M. Rozzo and their minor c...","43rd Street, Inc., Florida's Finest Seafood Co...","abdominal cramping, diarrhea; neurological pro...","the plaintiffs purchased and consumed fish, bl...",NaN,catherine brunson,names of the plaintiffs' expert not published ...,todd s. stewart of law offices of todd s. stew...
6,not published,"plaintiff v. emergency room physician, 2015 me...","January 5, 2015",ma,NaN,"300,000",the parties entered into a confidential settle...,Plaintiff,Emergency Room Physician,loss of testicle; severe abdominal pain,the plaintiff went to the defendant emergency ...,NaN,not published,not published,"russell x. pollock of bergstresser & pollock, ..."
7,not published,"plaintiff v. defendant medical facility, 2012 ...","May 4, 2012",ma,NaN,"2,000,000","following mediation, the parties agreed to a c...",Plaintiff,Defendant Medical Facility,death; sepsis,"the decedent, who suffered from gastric ulcers...",NaN,not published,not published,"max borten, sidney gorovitz of gorovitz & bort..."
8,not published,plaintiff v. defendant physician and nurse pra...,"Apr. 3, 2012",ma,NaN,"3,000,000","following mediation, the parties agreed to a c...",Plaintiff,Defendant Physician and Nurse Practitioner,bowel perforation; abdominal bruising,the defendant physician and nurse practitioner...,NaN,not published,not published,"andrew c. meyer, jr.; adam r. satin of lubin &..."
9,not published,"plaintiff v. defendant physician, 2012 medical...","Mar. 26, 2012",ma,NaN,"3,000,000",the parties entered into a confidential settle...,Plaintiff,Defendant Physician,ruptured abdominal aortic aneurysm; ischemic i...,the plaintiff alleged that the defendant physi...,NaN,not published,not published,"adam r. satin of lubin & meyer, boston, ma for..."


In [6]:
# fill in injury_df

# please check if you agree with the injury keywords!!

for x in range(0, len(data2)):
    a = data2[x]
    words = a.split()
    
    if words[0] !='AMOUNT:': #if the first word isn't AMOUNT, it's not a case but page number etc.
        injury_df.loc[x] = "NA" #fill entire row with NA
        
    else: #if it's a case
        lines = a.splitlines() #split the paragraph into lines
        
        for j in range(0, len(lines)): 
            w = str.lower(lines[j]).split() #convert text to lowercase and split into words   
            
            if w[0] == 'case:':
                injury_df.loc[x, 'case_name'] = ' '.join(w[1:len(w)])
                
            if (w[0] == 'specific' and w[1] == 'injury:') or (w[0] == 'specific' and w[1] == 'injuries:'):                
                
                # define keywords for injury
                keywords = ['abdominal', 'abdomen', 'torso',  'spleen', ' renal artery',  'upper extremity', 'peritonitis', 'CBD', 'bile duct', 'bile', 'appendix', 'intestine']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'abdominal'] = 0
                else:
                    injury_df.loc[x, 'abdominal'] = 1  
                    
                keywords = ['ankle']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'ankle'] = 0
                else:
                    injury_df.loc[x, 'ankle'] = 1           

                keywords = ['arm', 'forearm', 'ulna',  'brachial plexus']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'arm'] = 0
                else:
                    injury_df.loc[x, 'arm'] = 1  

                keywords = ['back']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'back'] = 0
                else:
                    injury_df.loc[x, 'back'] = 1          

                keywords = ['brain', 'cerebral', 'concussion',  'loss of consciousness', 'neurological','stroke']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'brain'] = 0
                else:
                    injury_df.loc[x, 'brain'] = 1          

                keywords = ['burn', 'burns']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'burn'] = 0
                else:
                    injury_df.loc[x, 'burn'] = 1  
                    
                keywords = ['diarrhea', 'bowel', 'colon']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'bowel'] = 0
                else:
                    injury_df.loc[x, 'bowel'] = 1                      

                keywords = ['chest', 'rib', 'ribs', 'lung', 'clavical', 'pulmonary', 'thoracic']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'chest'] = 0
                else:
                    injury_df.loc[x, 'chest'] = 1                          

                keywords = ['blood']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'circulatory'] = 0
                else:
                    injury_df.loc[x, 'circulatory'] = 1                      

                keywords = ['ear', 'hearing']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'ear'] = 0
                else:
                    injury_df.loc[x, 'ear'] = 1          

                keywords = ['elbow']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'elbow'] = 0
                else:
                    injury_df.loc[x, 'elbow'] = 1          

                keywords = ['eye', 'eyes', 'blindness']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'eye'] = 0
                else:
                    injury_df.loc[x, 'eye'] = 1              

                keywords = ['face', 'facial', 'forehead']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'face'] = 0
                else:
                    injury_df.loc[x, 'face'] = 1          

                keywords = ['foot', 'feet', 'heel']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'foot'] = 0
                else:
                    injury_df.loc[x, 'foot'] = 1          

                keywords = ['leg', 'tibia', 'tibial',  'sciatica']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'leg'] = 0
                else:
                    injury_df.loc[x, 'leg'] = 1          

                keywords = ['genital', 'testicle']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'genital'] = 0
                else:
                    injury_df.loc[x, 'genital'] = 1          

                keywords = ['hand', 'pinky', 'finger', 'thumb']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'hand'] = 0
                else:
                    injury_df.loc[x, 'hand'] = 1          

                keywords = ['head', 'headaches']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'head'] = 0
                else:
                    injury_df.loc[x, 'head'] = 1          

                keywords = ['heart', 'cardiac']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'heart'] = 0
                else:
                    injury_df.loc[x, 'heart'] = 1          

                keywords = ['hip']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'hip'] = 0
                else:
                    injury_df.loc[x, 'hip'] = 1          
                    
                keywords = ['knee']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'knee'] = 0
                else:
                    injury_df.loc[x, 'knee'] = 1       

                keywords = ['hepatic', 'liver']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'liver'] = 0
                else:
                    injury_df.loc[x, 'liver'] = 1                        

                keywords = ['mouth', 'lips', 'lip', 'jaw']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'mouth'] = 0
                else:
                    injury_df.loc[x, 'mouth'] = 1     

                keywords = ['neck', 'cervical']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'neck'] = 0
                else:
                    injury_df.loc[x, 'neck'] = 1          

                keywords = ['nose', 'nasal']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'nose'] = 0
                else:
                    injury_df.loc[x, 'nose'] = 1                                                 

                keywords = ['pelvic', 'pelvis', 'acetabular', 'pubic ramus', 'sacroiliac']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'pelvic'] = 0
                else:
                    injury_df.loc[x, 'pelvic'] = 1          

                keywords = ['shoulder', 'clavicle', 'rotator',  'sternoclavicular','subacromial decompression','acromioplasty']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'shoulder'] = 0
                else:
                    injury_df.loc[x, 'shoulder'] = 1     

                keywords = ['spinal', 'spine', 'disc', 'discs', 'vertebrae', 'radiculopathy', 'laminectomy', 'forminatomy', 'diskectomy', 'T8', 'lumbar', 'herniation']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'spinal'] = 0
                else:
                    injury_df.loc[x, 'spinal'] = 1          

                keywords = ['thigh', 'femur', 'shaft']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'thigh'] = 0
                else:
                    injury_df.loc[x, 'thigh'] = 1    
                    
                keywords = ['sepsis', 'hemiparesis', 'multiple fractures', 'internal injuries']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'whole_body'] = 0
                else:
                    injury_df.loc[x, 'whole_body'] = 1                      

                keywords = ['wrist', 'carpel tunnel']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'wrist'] = 0
                else:
                    injury_df.loc[x, 'wrist'] = 1  
                    
                keywords = ['pancreas', 'pancreatitis']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'pancreas'] = 0
                else:
                    injury_df.loc[x, 'pancreas'] = 1                        

                keywords = ['psychological', 'cognitive', 'memory', 'PTSD', 'traumatic stress', 'autism']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'psychological'] = 0
                else:
                    injury_df.loc[x, 'psychological'] = 1          

                keywords = ['death']
                z = [i for e in keywords for i in w if e in i]
                if len(z) == 0:
                    injury_df.loc[x, 'death'] = 0
                else:
                    injury_df.loc[x, 'death'] = 1                      
        

In [7]:
injury_df

,case_name,abdominal,ankle,arm,back,brain,burn,bowel,chest,circulatory,...,nose,pelvic,shoulder,spinal,thigh,whole_body,wrist,pancreas,psychological,death
0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,"jeremy freeman v. southern medical group, 2014...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,"plaintiff v. defendant doctor, 2016 medical li...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"jackson v. usa, 2012 medical litig. alert lexi...",1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5,"todd rozzo, valerie m. rozzo and their minor c...",1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"plaintiff v. emergency room physician, 2015 me...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"plaintiff v. defendant medical facility, 2012 ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8,plaintiff v. defendant physician and nurse pra...,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"plaintiff v. defendant physician, 2012 medical...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Combine data frames

data_all = pd.merge(finaldf, injury_df, on="case_name")

In [9]:
# View final data frame

data_all

,case_number,case_name,date,state,county,amount,pertinent_info,plaintiff,defendant,injury_list,...,nose,pelvic,shoulder,spinal,thigh,whole_body,wrist,pancreas,psychological,death
0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,cv-2007-000134,"jeremy freeman v. southern medical group, 2014...",October 2014,al,NaN,"1,150,000",NaN,Jeremy Freeman,Southern Medical Group,cardiac arrest; punctured abdominal wall; death,...,0,0,0,0,0,0,0,0,0,1
5,not published,"plaintiff v. defendant doctor, 2016 medical li...",January 2016,ca,NaN,"1,000,000",NaN,Plaintiff,Defendant Doctor,injury to common bile duct and hepatic artery,...,0,0,0,0,0,0,0,0,0,0
6,3:09-cv-00026,"jackson v. usa, 2012 medical litig. alert lexi...",July 2012,fl,NaN,"750,000",the case was settled prior to trial. the plain...,Jackson,USA,peritonitis; sepsis; stroke; permanent cogniti...,...,0,0,0,0,0,1,0,0,1,0
7,2008 ca 028502,"todd rozzo, valerie m. rozzo and their minor c...","Aug. 21, 2012",fl,NaN,"320,000",NaN,"Todd Rozzo, Valerie M. Rozzo and their minor c...","43rd Street, Inc., Florida's Finest Seafood Co...","abdominal cramping, diarrhea; neurological pro...",...,0,0,0,0,0,0,0,0,0,0
8,not published,"plaintiff v. emergency room physician, 2015 me...","January 5, 2015",ma,NaN,"300,000",the parties entered into a confidential settle...,Plaintiff,Emergency Room Physician,loss of testicle; severe abdominal pain,...,0,0,0,0,0,0,0,0,0,0
9,not published,"plaintiff v. defendant medical facility, 2012 ...","May 4, 2012",ma,NaN,"2,000,000","following mediation, the parties agreed to a c...",Plaintiff,Defendant Medical Facility,death; sepsis,...,0,0,0,0,0,1,0,0,0,1


In [10]:
# Save to computer
# os.chdir('/Users/Felicia/Documents/ComputationalLaw/')
os.chdir('/Users/thymol/Desktop/Comp/pred-case-outcomes')

data_all.to_csv('combined_WIW_data.csv', index=False)